**Вакансии с HeadHunter**

## Установка

In [ ]:
!pip install fake_useragent 
# установка fake-useragent для генерации фейкового юзер-агента 
# рандомное сочетание операционной системы, спецификаций и версии браузера
!pip install selenium 
# установка библиотеки selenium для "встраиваться в браузер и работать в нем как пользователь"
!pip install tqdm

## Код

In [1]:
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random
from tqdm import tqdm
import pandas as pd

position = '+'.join(input('Должность: ' ).split( ))

infa = [ ]
for p in tqdm(range(0,36)):
    url = f'https://hh.ru/search/vacancy?text={position}&from=suggest_post&area=1&page={p}'
    page = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(page.text, 'html')
    vacancies = soup.find_all('div', {'class' : 'serp-item'})
    for vacancy in vacancies:
        try:
            infa.append({'name': vacancy.find('a', {'class' : 'serp-item__title'}).text,
                         'link': vacancy.h3.a.get('href'),
                         'employer': vacancy.find('a', {'class' : 'bloko-link bloko-link_kind-tertiary'}).text,
                         'address': vacancy.find('div', {'data-qa' : 'vacancy-serp__vacancy-address'}).text,
                         'salary': vacancy.find('span', {'class' : 'bloko-header-section-3'})})
        except:
            pass 
    time.sleep(random.random()*2)
df = pd.DataFrame(infa)
print(df.shape)
df.head(10)

Должность: бизнес аналитик


100%|██████████| 36/36 [00:58<00:00,  1.63s/it]

(638, 5)


,name,link,employer,address,salary
0,Старший кредитный аналитик (малый бизнес),https://hh.ru/vacancy/66853831?from=vacancy_se...,Сбер для экспертов,Москва,"[от , , 146 400, , , , руб.]"
1,Бизнес-аналитик B2C,https://hh.ru/vacancy/69541498?from=vacancy_se...,Simple. Коммерческая дирекция B2С,"Москва, Минская",None
2,Аналитик (TMS AXELOT),https://hh.ru/vacancy/71305599?from=vacancy_se...,Франчайзинговая розничная сеть Галамарт,"Москва, Парк культуры и еще 2",None
3,Аналитик 1С,https://hh.ru/vacancy/71029120?from=vacancy_se...,Франчайзинговая розничная сеть Галамарт,"Москва, Парк культуры и еще 2",None
4,Системный/Бизнес Аналитик,https://hh.ru/vacancy/71234973?from=vacancy_se...,ООО Датаномика,"Москва, Кутузовская","[200 000 – 300 000 , , руб.]"
5,Бизнес-аналитик,https://hh.ru/vacancy/71416002?from=vacancy_se...,Глобал Нетворкс,"Москва, Нагатинская","[от , , 200 000, , , , руб.]"
6,Бизнес-аналитик (банковское направление),https://hh.ru/vacancy/69177899?from=vacancy_se...,HFLabs,"Москва, Парк культуры","[250 000 – 300 000 , , руб.]"
7,Системный аналитик/Бизнес-аналитик (стажер),https://hh.ru/vacancy/71047871?from=vacancy_se...,ООО Фортис Технологии,"Москва, Полежаевская","[60 000 – 90 000 , , руб.]"
8,Аналитик (бизнес\системный) направления по раб...,https://hh.ru/vacancy/70986979?from=vacancy_se...,HFLabs,Москва,"[230 000 – 270 000 , , руб.]"
9,Бизнес - аналитик (стажер),https://hh.ru/vacancy/70958478?from=vacancy_se...,Лаборатория информационных технологий,Москва,"[23 500 – 50 000 , , руб.]"


## selenium

In [83]:
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options

options = Options()
options.headless = True # без физического открытия браузера

browser = webdriver.Edge('C:/Users/khsinchuk/Documents/Обучение/edgedriver_win64/msedgedriver.exe', options=options)
# Как только ключ executable_path устарел, вы должны использовать экземпляр Service() класса
# см. https://stackoverflow.com/questions/69918148/deprecationwarning-executable-path-has-been-deprecated-please-pass-in-a-servic
# см. https://itsmycode.com/executable-path-has-been-deprecated/

url = 'https://hh.ru/?hhtmFrom=main'
browser.get(url)
field = browser.find_element('xpath','//*[@id="a11y-search-input"]') # новый сиснтаксис вместо find_element_by_xpath
position = list(input('Должность: ' ))
i = 0
for letter in position:
    field.send_keys(position[i])
    i += 1
browser.implicitly_wait(random.random()*2)
submit = browser.find_element('xpath','//*[@id="HH-React-Root"]/div/div[3]/div[1]/div[1]/div/div/div[2]/div/form/div/div[2]/button')
submit.click()

infa = [ ]
i = 0
while True:
    page = browser.page_source # cохраним страницу с результатами в переменную
    soup = BeautifulSoup(page, 'html')
    vacancies = soup.find_all('div', {'class' : 'serp-item'})
    for vacancy in vacancies:
        try:
            infa.append({'name': vacancy.find('a', {'class' : 'serp-item__title'}).text,
                         'link': vacancy.h3.a.get('href'),
                         'employer': vacancy.find('a', {'class' : 'bloko-link bloko-link_kind-tertiary'}).text,
                         'address': vacancy.find('div', {'data-qa' : 'vacancy-serp__vacancy-address'}).text,
                         'salary': vacancy.find('span', {'class' : 'bloko-header-section-3'})})
        except:
            pass
    try:    
        if " " in position: # на сайте разные элементы в зависимости от наличия пробела
            next_p = browser.find_element('xpath',
                                          '//*[@id="HH-React-Root"]/div/div[3]/div[1]/div/div[3]/div[2]/div[2]/div/div[5]/div/a/span')
        else:
            next_p = browser.find_element('xpath',
                                          '//*[@id="HH-React-Root"]/div/div[3]/div[1]/div/div[2]/div[2]/div[2]/div/div[5]/div/a/span')
        next_p.click()
    except:
        break

browser.close()

df = pd.DataFrame(infa)
print(df.shape)
df.head(10)


C:\Users\khsinchuk\AppData\Local\Temp\ipykernel_22516\297975079.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Edge('C:/Users/khsinchuk/Documents/Обучение/edgedriver_win64/msedgedriver.exe', options=options)


Должность: руководитель по оценке персонала
(1110, 5)


,name,link,employer,address,salary
0,HR Director (Руководитель отдела построения),https://hh.ru/vacancy/71821725?from=vacancy_se...,kt.team,Москва,"[250 000 – 400 000 , , руб.]"
1,HR-generalist / Руководитель отдела развития и...,https://hh.ru/vacancy/71617422?from=vacancy_se...,SK Design (СК ДИЗАЙН),Москва,"[от , , 130 000, , , , руб.]"
2,"Директор по персоналу (HRD, HRBP, средний бизн...",https://hh.ru/vacancy/71758173?from=vacancy_se...,NSHR Рекрутинг и не только,Москва,"[до , , 400 000, , , , руб.]"
3,"Руководитель направления подбора, адаптации и ...",https://hh.ru/vacancy/70962424?from=vacancy_se...,ООО АТОМСТРОЙРЕЗЕРВ,Москва,"[от , , 150 000, , , , руб.]"
4,Руководитель отдела обучения и развития персонала,https://hh.ru/vacancy/71903500?from=vacancy_se...,ООО РОСТАР,Москва,"[150 000 – 200 000 , , руб.]"
5,Руководитель департамента по работе с персоналом,https://hh.ru/vacancy/71416634?from=vacancy_se...,ООО Агентство судебного взыскания,Москва,"[от , , 150 000, , , , руб.]"
6,HR-директор,https://hh.ru/vacancy/71934652?from=vacancy_se...,ИП Панфилов Роман Игоревич,Москва,"[70 000 – 130 000 , , руб.]"
7,Руководитель направления оценки и развития пер...,https://hh.ru/vacancy/71930023?from=vacancy_se...,Управление персоналом,Москва,None
8,Руководитель отдела подбора массового персонала,https://hh.ru/vacancy/71976123?from=vacancy_se...,ООО ГК АРОСТА,Москва,None
9,HR Generalist / Руководитель отдела персонала ...,https://hh.ru/vacancy/71889284?from=vacancy_se...,Специальные стали и сплавы,Москва,"[100 000 – 200 000 , , руб.]"
